In [2]:
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import numpy as np 
import matplotlib.pyplot as plt
import os

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.layers import Embedding
import seaborn as sns
pd.set_option('display.max_colwidth', -1)

<ipython-input-2-b97772079596>:14: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
df = pd.read_json('../../../data/data.json', lines=True)
df.drop(['authors', 'link', 'date'], axis = 1, inplace = True) 
df.head()

,category,headline,short_description
0,CRIME,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV",She left her husband. He killed their children. Just another day in America.
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song,Of course it has a song.
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork,The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog,"The ""Dietland"" actress said using the bags is a ""really cathartic, therapeutic moment."""


In [3]:
categories = df['category'].value_counts().index

def groupper(grouplist,name):
    for ele in categories:
        if ele in grouplist:
            df.loc[df['category'] == ele, 'category'] = name

In [4]:
groupper( grouplist= ['WELLNESS', 'HEALTHY LIVING','HOME & LIVING','STYLE & BEAUTY' ,'STYLE'] , name =  'LIFESTYLE AND WELLNESS')

groupper( grouplist= [ 'PARENTING', 'PARENTS' ,'EDUCATION' ,'COLLEGE'] , name =  'PARENTING AND EDUCATION')

groupper( grouplist= ['SPORTS','ENTERTAINMENT' , 'COMEDY','WEIRD NEWS','ARTS'] , name =  'SPORTS AND ENTERTAINMENT')

groupper( grouplist= ['TRAVEL', 'ARTS & CULTURE','CULTURE & ARTS','FOOD & DRINK', 'TASTE'] , name =  'TRAVEL-TOURISM & ART-CULTURE')

groupper( grouplist= ['WOMEN','QUEER VOICES', 'LATINO VOICES', 'BLACK VOICES'] , name =  'EMPOWERED VOICES')

groupper( grouplist= ['BUSINESS' ,  'MONEY'] , name =  'BUSINESS-MONEY')

groupper( grouplist= ['THE WORLDPOST' , 'WORLDPOST' , 'WORLD NEWS'] , name =  'WORLDNEWS')

groupper( grouplist= ['ENVIRONMENT' ,'GREEN'] , name =  'ENVIRONMENT')

groupper( grouplist= ['TECH', 'SCIENCE'] , name =  'SCIENCE AND TECH')

groupper( grouplist= ['FIFTY' , 'IMPACT' ,'GOOD NEWS','CRIME'] , name =  'GENERAL')

groupper( grouplist= ['WEDDINGS', 'DIVORCE',  'RELIGION','MEDIA'] , name =  'MISC')

In [5]:
print("We have a total of {} categories now".format(df['category'].nunique()))
df['category'].value_counts()

We have a total of 12 categories now


LIFESTYLE AND WELLNESS          40619
POLITICS                        32739
SPORTS AND ENTERTAINMENT        30296
TRAVEL-TOURISM & ART-CULTURE    20578
EMPOWERED VOICES                15461
PARENTING AND EDUCATION         14780
MISC                            12448
GENERAL                         9663 
WORLDNEWS                       8420 
BUSINESS-MONEY                  7644 
SCIENCE AND TECH                4260 
ENVIRONMENT                     3945 
Name: category, dtype: int64

In [6]:
df2 = df.copy()

In [7]:
print(len(df2[df2['short_description'] == ""]))

19712


In [8]:
df2['text'] = df2['headline'].astype(str)+"-"+df2['short_description']
df2.drop(columns =['headline','short_description'],axis = 1, inplace=True)
df2.astype(str)
df2

,category,text
0,GENERAL,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV-She left her husband. He killed their children. Just another day in America."
1,SPORTS AND ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song-Of course it has a song.
2,SPORTS AND ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57-The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.
3,SPORTS AND ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork-The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.
4,SPORTS AND ENTERTAINMENT,"Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog-The ""Dietland"" actress said using the bags is a ""really cathartic, therapeutic moment."""
...,...,...
200848,SCIENCE AND TECH,RIM CEO Thorsten Heins' 'Significant' Plans For BlackBerry-Verizon Wireless and AT&T are already promoting LTE devices including smartphones and tablets from RIM's rivals. RIM's first
200849,SPORTS AND ENTERTAINMENT,"Maria Sharapova Stunned By Victoria Azarenka In Australian Open Final-Afterward, Azarenka, more effusive with the press than normal, credited her coach of two years, Sam Sumyk, for his patient"
200850,SPORTS AND ENTERTAINMENT,"Giants Over Patriots, Jets Over Colts Among Most Improbable Super Bowl Upsets Of All Time (VIDEOS)-Leading up to Super Bowl XLVI, the most talked about game could end up being one that occurred a few years ago. After all"
200851,SPORTS AND ENTERTAINMENT,Aldon Smith Arrested: 49ers Linebacker Busted For DUI-CORRECTION: An earlier version of this story incorrectly stated the location of KTVU and the 2011 league leader in sacks


In [9]:
from sklearn.utils import shuffle
df2 = shuffle(df2)
df2.reset_index(inplace=True, drop=True) 

In [13]:
X = df2['text']
Y= df2['category']
#80% to train , 10% for validation , 10% for testing


X_train, X_val, y_train, y_val = train_test_split(X,Y, test_size=0.2, random_state=42)
X_val, X_test , y_val, y_test= train_test_split(X_val,y_val, test_size=0.5, random_state=42)

In [14]:
vocab_size =20000
max_length = 150
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [15]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

word_index = tokenizer.word_index

X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train,maxlen= max_length,padding=padding_type, truncating=trunc_type)
y_train = np.asarray(y_train)
y_train = pd.get_dummies(y_train)

X_val = tokenizer.texts_to_sequences(X_val)
X_val = pad_sequences(X_val,maxlen= max_length,padding=padding_type, truncating=trunc_type)
y_val = np.asarray(y_val)
y_val = pd.get_dummies(y_val)

train_set = np.array(X_train)
val_set = np.array(X_val)

train_label = np.array(y_train)
val_label = np.array(y_val)


y_test = pd.get_dummies(y_test)
y_test = np.asarray(y_test)
y_test = np.argmax(y_test,axis=1)   #this would be our ground truth label while testing

print(train_set.shape)
print(train_label.shape)


print(val_set.shape)
print(val_label.shape)

(160682, 150)
(160682, 12)
(20085, 150)
(20085, 12)


In [20]:
path_to_glove_file =  '../../../../../glove.6B.100d.txt'

In [22]:
#Initialising the embedding matrix with glove vec embeddings

num_tokens = len(tokenizer.word_index.items()) + 2
embedding_dim = 100
hits = 0
misses = 0


embeddings_index = {}
with open(path_to_glove_file, encoding="utf8" ) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))


# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Found 400001 word vectors.
Converted 66004 words (38804 misses)


In [26]:
early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                            patience=3, min_delta=0.0001)


tf.keras.backend.clear_session()
embed_size = 100
model = keras.models.Sequential([
                                 
        Embedding(num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        mask_zero=True,input_shape=[None],trainable=False),
        keras.layers.Bidirectional(keras.layers.LSTM(32, dropout = 0.4)),
        keras.layers.Dense(12, activation="softmax")
            
        ])


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         10481000  
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                34048     
_________________________________________________________________
dense (Dense)                (None, 12)                780       
Total params: 10,515,828
Trainable params: 34,828
Non-trainable params: 10,481,000
_________________________________________________________________


In [27]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
history = model.fit( train_set,train_label,
                     batch_size = 32,
                     steps_per_epoch=len(X_train) // 32, 
                     validation_data = (val_set , val_label),
                     validation_steps = len(val_set)//32, epochs=20,
                     callbacks=  early_stop )

Epoch 1/20
5021/5021 [==============================] - 345s 69ms/step - loss: 1.2954 - accuracy: 0.5842 - val_loss: 1.0726 - val_accuracy: 0.6552
Epoch 2/20
5021/5021 [==============================] - 364s 73ms/step - loss: 1.1080 - accuracy: 0.6450 - val_loss: 1.0247 - val_accuracy: 0.6674
Epoch 3/20
5021/5021 [==============================] - 369s 73ms/step - loss: 1.0676 - accuracy: 0.6569 - val_loss: 0.9992 - val_accuracy: 0.6755
Epoch 4/20
5021/5021 [==============================] - 376s 75ms/step - loss: 1.0439 - accuracy: 0.6652 - val_loss: 0.9884 - val_accuracy: 0.6824
Epoch 5/20
5021/5021 [==============================] - 364s 72ms/step - loss: 1.0295 - accuracy: 0.6684 - val_loss: 0.9741 - val_accuracy: 0.6887
Epoch 6/20
5021/5021 [==============================] - 371s 74ms/step - loss: 1.0213 - accuracy: 0.6712 - val_loss: 0.9643 - val_accuracy: 0.6902
Epoch 7/20
5021/5021 [==============================] - 396s 79ms/step - loss: 1.0117 - accuracy: 0.6738 - val_loss: 0

In [41]:
model.save('./best_model2')

In [3]:
model3 = keras.models.load_model('./best_model')


In [4]:
text_1 = 'Pelosi: Biden Can’t Cancel Student Debt '
text_2 = "Feds Take on Student Homelessness"
print(model3.predict([text_1, text_2]))

UnimplementedError:  Cast string to float is not supported
	 [[node sequential/Cast (defined at <ipython-input-4-97feccca2661>:3) ]] [Op:__inference_predict_function_46558]

Function call stack:
predict_function


In [6]:
np.testing.assert_allclose(model3.predict('Pelosi: Biden Can’t Cancel Student Debt ')


SyntaxError: unexpected EOF while parsing (<ipython-input-6-2eacee4aa4da>, line 2)